In [208]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
import tensorflow as tf
import statsmodels.api as sm

# Reading in the Data

In [98]:
%%time
data = pd.read_csv('data.csv')

<decorator-gen-54>:2: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 23.9 s, sys: 4.89 s, total: 28.8 s
Wall time: 29.6 s


In [99]:
data['game_date'] = pd.to_datetime(data['game_date'])

# Creating a target -> Win or not

In [100]:
data = data[data['score_differential'].isna() == False] # Dropping NaN values from score_differential

In [101]:
lookup_wins = data.groupby('game_id').tail(1)
lookup_wins = lookup_wins.set_index("game_id")
lookup_wins["wonGame"] = lookup_wins["score_differential"] > 0

In [102]:
lookup_wins = lookup_wins[["wonGame"]]

In [103]:
data = data.merge(lookup_wins, right_on=["game_id"], left_on=["game_id"])

Home Field Advantage %

In [104]:
lookup_wins["wonGame"].value_counts()[True] / lookup_wins["wonGame"].value_counts().sum()

0.6064924782264449

In [105]:
data["game_seconds_remaining"]

0         3593.0
1         3556.0
2         3515.0
3         3507.0
4         3496.0
           ...  
433947      64.0
433948      63.0
433949      58.0
433950      38.0
433951      35.0
Name: game_seconds_remaining, Length: 433952, dtype: float64

# Cleaning the Data

In [106]:
data["down"].dropna(inplace=True)

# Relevant Data

In [131]:
relevant_data = data[["score_differential", "game_seconds_remaining", "wonGame"]]


In [132]:
# relevant_data = relevant_data[relevant_data["game_seconds_remaining"]

In [133]:
relevant_data.dropna(inplace=True)

<ipython-input-133-5933214d0cd8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_data.dropna(inplace=True)


In [174]:
relevant_data = relevant_data[abs(relevant_data["score_differential"]) > 21]

In [175]:
X_train, X_test, y_train, y_test = train_test_split(relevant_data.drop(columns=["wonGame"]), relevant_data["wonGame"], test_size=0.2, random_state=42, stratify=relevant_data["wonGame"])

In [176]:
train, test = train_test_split(relevant_data, test_size=0.2, random_state=42, stratify=relevant_data["wonGame"])

# Modeling

In [177]:
class custom_model():
    def fit(self, X, y):
        self.X = X
        self.y = y

    def predict(self, X):
        return X.apply(lambda x: 1 if x > 0 else 0)

IndentationError: expected an indented block (<ipython-input-177-073e21a74841>, line 3)

In [209]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)
model.score(X_test, y_test)

CPU times: user 1.62 s, sys: 39 ms, total: 1.66 s
Wall time: 1.67 s


0.6673790557606373

In [210]:
results = model.predict(X_test)

In [211]:
model.predict_proba([[-40, 30]])

array([[0.87673512, 0.12326488]])

In [212]:
np.unique(results, return_counts=True)[1]

array([ 162, 4985])